# Querying our GraphQL API

The PostgreSQL database exposes a GraphQL API that one could query directly. Usually it's more convenient to use the caching interface that we use in the flatsurvey scripts themselves to speed up surveys.

In [1]:
from flatsurvey.cache import GraphQL
cache = GraphQL()

## Interval Exchange Transformations which remained Undetermined

In [2]:
from flatsurvey.jobs import UndeterminedIntervalExchangeTransformation

If we are not interested in the undetermined components but just in the Interval Exchange Transformations that they correspond to, we can pull these directly from the database.

In [3]:
undetermined = cache.results(
    job=UndeterminedIntervalExchangeTransformation,
    filter="intervals: { lessThan: 9 } degree: { lessThan: 5 }")

The easiest way to get the original `pyintervalxt` objects back from this iterable of rows is to call `.results()`. Note that this gives a lazy iterable, so converting that to a list might result in a very substantial number of downloads. It is usually best to loop of the results instead.

In [4]:
iets = undetermined.results()

We zoom in on the first IET returned:

In [5]:
iet = next(iets)
iet

[a: (-2639487616593692358066855581592185924618270835741586414871941205927606619638311916472282790225563603089371645696/3*c^3 + 263991386043436930863846661657435256879331459855940382016061134637805669849439332614169772877834141125860210048/3*c^2 + 28579959189761353644528317576652975865389276294845731629568031083509321325902759493325643941787212154535734026368/9*c - 911185582088210381587160815590341008515670886431230703455744040456984472188468204618700513586601950212952928000/3 ~ 5.2386539e-150)] [b: (-61104658941820264073880185364287257217570439332079248449400636810781171511335935951296916817433031060920977300608/9*c^3 + 679050295231167782934678272167015020369799275103141577405980521765328279735634107891543069414289248164108879744*c^2 + 73514632489974214405726442071519246912742976359396853172059831983209067422967914142011670300824887413511565401344/3*c - 7031375316453419491414977073081179780654635543160103945412014614853758921104404931805668268840971769100394546560/3 ~ 6.3274550e-152)] 

Sanity check that it should have been discarded by Boshernitzan's algorithm right away:

In [6]:
iet.boshernitzanNoPeriodicTrajectory()

False

We already did quite a few induction steps on these; how many can be seen from the `invocation`, namely it's the `--limit` of `flow-decomposition`. When there is no such limit, it is just the default which currently is 256.

In [7]:
next(iter(undetermined))

{'id': '4d166a1e-685a-11eb-bcf5-061402de139a',
 'timestamp': datetime.datetime(2021, 2, 6, 9, 4, 21, 452139, tzinfo=tzutc()),
 'data': {'lot': 7506735730201822000,
  'degree': 4,
  'result': <function Results._resolve.<locals>.restore at 0x7fee27a7a440>,
  'command': ['undetermined-iet'],
  'intervals': 8,
  'invocation': ['/dev/shm/ruth/flatsurvey/flatsurvey/worker/__main__.py',
   'log',
   '--output=ngon-1-3-3-3.log',
   'graphql',
   'orbit-closure',
   'undetermined-iet',
   'flow-decompositions',
   '--limit=1024',
   'pickle',
   '--base64',
   'eJyVVgd4FFUQDqEvCCIIVsR+qLkUjIJYiDQhcuIAepa4bvZebtfs7ebf3QtEjaC4t8TeK/bee++99957772X2XLkEgLil+97t9P/mTczL4vK1SZDcZ283SrakvzTpKjCSZpZy3SkFJ8SyhrQK6E6SlYkbd3MOknddEVW2FJOaRZykUB5uldZWVk1envoQ+irRfQ49PPQnzBAK9f4ayBBir4GEQaLdH9WERWK6eoqVusSxMznGoUtN+nCyHQhZGGInDBdKRUypwW8qRFLVhody8i7QsKQyJfj2nnVzdsiyXm5lt0mZYUpbF2VY1rOmy262mywydDEimwMy2rOt8hZw2pUDAmrpycw7FXBmizBKLcK29EtswbDPKxBGF4/sL68gBFdsrYVl1UUIzKXKCZDBxLWHEsYWareYhltppXTFaPkU+50YvDtSLNXK

Let's try harder by running more iterations on top of the previous ones:

In [8]:
iet.induce(10000R)

LIMIT_REACHED

In [9]:
iet.induce(10000R)

NON_SEPARATING_CONNECTION()

## Surfaces with non-dense Orbit Closure

In [ ]:
from flatsurvey.jobs import OrbitClosure

We pull all OrbitClosures from the cache where we could not determine that the orbit closure was dense. Since we can currently never determine that the orbit closure is *not* dense, these are the ones where the search was inconclusive, that is, it reported `None` which turns to `null` in this API.

In [ ]:
nondense = cache.query(job=OrbitClosure, result_filter="dense: { isNull: true }")

The underlying objects from the database are in `.nodes()`. Each of these nodes corresponds to one `OrbitClosure` computation. The corresponding surface is a pickle that can be recovered with such a call:

In [ ]:
nondense.nodes()[0]['surface']()

Let's recover all the surfaces and remove duplicates:

In [ ]:
surfaces = set(node['surface']() for node in nondense.nodes())

We remove all the surfaces that we already know about as encoded by `.reference()`.

In [ ]:
surfaces = set(surface for surface in surfaces if surface.reference() is None)

Note that this set might appear to contain duplicates since e.g. two quadrilaterals with differently chosen random lengths are printed in the same way. Let's remove these "duplicates" as well.

In [ ]:
surfaces = set(str(surface) for surface in surfaces)

Some of the orbit closure computations might not have found the full dimension of the orbit closure because they did not search deep enough. Let's recover all the computations for our surfaces and remove the surfaces where some run reported a dense orbit closure.

We could use `cache.query` again for this. To determine the cached results for a fixed surface, `cache.results()` provides a wrapper around `cache.query` for this.

In [ ]:
orbit_closures = {
    name: cache.results(job=OrbitClosure, surface=name) for name in surfaces
}

We filter out all the surfaces where some run determined that the orbit closure was dense.

This could be done by comparing `node['dense']` for each node in `.nodes()`. Here we use `.reduce()` which calls into `OrbitClosure` to combine the results of several runs.

In [ ]:
orbit_closure = {
    name: cached for (name, cached) in orbit_closures.items() if cached.reduce() is None
}

In [ ]:
orbit_closures

Let's zoom in on one particular surface:

In [ ]:
orbit_closures = orbit_closures['Ngon([3, 4, 13])']

Again, `.nodes()` contains the raw data stored in the database.

In [ ]:
orbit_closures.nodes()

To unpickle all the `['result']` objects at once, we can use `.results()`:

In [ ]:
orbit_closures.results()

## Flow Decompositions with Undetermined Components

In [ ]:
from flatsurvey.jobs import FlowDecompositions

We pull all the flow decompositions coming from triangles from the database that had undetermined components.

Note we need to `limit` the result to the most recent decompositions since there are way too many (>500k) in the database to download them all:

In [ ]:
undetermined = cache.query(job=FlowDecompositions, surface_filter="vertices: { equalTo: 3 }", result_filter="undetermined: { notEqualTo: 0 }", limit=32)

Let's zoom in on one such decomposition:

In [ ]:
undetermined = undetermined.nodes()[0]

Unforfunately, `flatsurf::FlowDecomposition` cannot be pickled currently so trying to recover the pickle fails:

In [ ]:
undetermined['result']

But we can recover the surface and the direction that were used:

In [ ]:
import pyflatsurf
surface = undetermined['surface']()
direction = undetermined['orientation']()

In [ ]:
surface

And from this we can recover the decomposition:

In [ ]:
pyflatsurf.flatsurf.makeFlowDecomposition(surface.flat_triangulation(), direction)